# Data Cleanup
Now let's begin the process of cleaning up the string data we scraped from Glassdoor.

In [ ]:
# libraries needed
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_rows', 1000)
pd.options.mode.chained_assignment = None

In [ ]:
# get some information on the saved data
file_name = r"C:\Users\Tineash\Projects\Glassdoor_webscraper\Data\Data_analyst_dataset.pkl" # enter the filepath between the quotes
data = pd.read_pickle(file_name)
data["Location"].value_counts()
#clean up job title lengths to cut off at a certain point (avoids empty cells when doing downstream work)

In [ ]:
#We see some NaN values, so let's confirm they are recognized as nulls
data.info()

Above, we can see that some columns are fully populated, such as 'Job Title', but others have nulls. I do see some black values where there should be NaNs, so I will run through the document and replace empty cells with NaN. I will want to convert the year founded into years existing. I have to clean up some duplicates I see, convert the salary to a float and remove non numeric characters, and clean up some trailing characters at the end of the company names (/n#). It would be useful to have the location be split into city and states. I may want to clean up the string for type of ownership into just Private vs Public, but I'll run through it to confirm. Finally, I will have to review the revenue data and convert the Unknown/NA into nulls, then determine if the information can be used. **A significant and fun list!**

In [ ]:
#Replace empty cells with NaN
#r = raw string. ^ = start of line $ = end of line \s* = any length of string (accounts for whitespace)
data = data.replace(r'^\s*$', np.nan, regex=True) # we use regex to check the cell expression and see if it matches the input

In [ ]:
# checking columns for nulls
data.isnull().sum()

In [ ]:
#Remove features where the salary is null,since that doesnt help us
data_cleaned = data.dropna(axis=0)
data_cleaned.isnull().sum()
data_cleaned.info()

In [ ]:
# separate hourly rows from salary rows
data_cleaned = pd.DataFrame(data = data_cleaned) # convert the slice to a pandas dataframe to work with it
data_cleaned['Average Hourly Rate'] = data_cleaned["Salary Average"].apply(lambda x: 1 if '/hr' in x.lower() else 0)
data_cleaned = data_cleaned.reset_index(drop=True)
data_cleaned[['Salary Minimum','Salary Average']].value_counts()


We can see that there appears to be average hourly rate data in here. Let's split those out so we can compare the hourly rates to the salary rates givin when we analyze the data. There is also a strange /mo data grouping with 6 entries, but the salary is way out of range for the job title, so I'm assuming that is meant to be /year. I'll treat it as such and scrape out the data.

In [ ]:
#clean up Salary min/max/average strings to only have numeric strings, then convert to float
# remove $, ',',(/yr (est.)

#First, lets clean up the Average Salary
salary_avg = data_cleaned['Salary Average'].apply(lambda x: x.split("/")[0])
salary_avg = salary_avg.apply(lambda x: x.replace('$', '').replace(',',''))
data_cleaned['Salary Average'] = salary_avg

#Now the minimum salary
salary_min = data_cleaned['Salary Minimum'].apply(lambda x:x.replace('$', '').replace('K','').replace('/hr', '').replace('/mo',''))
data_cleaned['Salary Minimum']=salary_min


#Now the maximum salary
salary_max = data_cleaned['Salary Maximum'].apply(lambda x:x.replace('$', '').replace('K','').replace('/hr', '').replace('/mo',''))
data_cleaned['Salary Maximum']=salary_max
data_cleaned[["Salary Minimum", "Salary Maximum","Salary Average"]].value_counts()
#data_cleaned = data_cleaned.replace(r'^\s*$', np.nan, regex=True)
#data_cleaned.isnull().sum()

Now lets convert the hourly values to yearly salary.

In [ ]:
# convert hourly salary to yearly salary
# convert $/hr to $/year and replace cells with the yearly estimate
def hr_to_year(i):
    i = int(float(i)) # convert the string to a float
    i = i*40*52
    #print("A rate of $", i,"/hr will be a salary of $", int(salary),"/yr.")
    return i
#convert hourly salary to yearly salary
data_cleaned["Salary Average"] = data_cleaned["Salary Average"].astype(str).apply(lambda x: hr_to_year(x)/1000 if x.find('.') != -1 else (float(x)/1000))
#data_cleaned["Converted Salary"] = data_cleaned["Converted Salary"]/1000
#convert min/max hourly rates to yearly rates
data_cleaned["Salary Minimum"] = data_cleaned["Salary Minimum"].astype(str).apply(lambda x: hr_to_year(x)/1000 if x.find('.') != -1 else x)
data_cleaned["Salary Maximum"] = data_cleaned["Salary Maximum"].astype(str).apply(lambda x: hr_to_year(x)/1000 if x.find('.') != -1 else x)
data_cleaned[["Salary Minimum", "Salary Maximum", "Salary Average"]].value_counts()


With the hourly rate converted to yearly salary and a column marking which rows had an hourly value, we can now move on to some simplier tasks, such as converting data types and cleaning up some strings.

In [ ]:
# remove trailing 5 characters from Company Name strings
data_cleaned['Company Name'] = data_cleaned["Company Name"].apply(lambda x: x.split('\n', 1)[0] if x.find('\n') != -1 else x)
data_cleaned['Company Name'].head(15)


In [ ]:
# split location to city and state


#str(string_check.iloc[0]).split(',')[1]
#a = 0
#for i in DA_data_cleaned["Location"]:
#    if ',' in str(DA_data_cleaned["Location"].iloc[i]):
#        DA_data_cleaned["City"] = str(DA_data_cleaned["Location"].iloc[i]).split(',')[0]
#        DA_data_cleaned["State"] = str(DA_data_cleaned["Location"].iloc[i]).split(',')[1]
#        a += 1
#    else:
#        DA_data_cleaned["City"] = "Remote"
#        DA_data_cleaned["State"] = "Remote"
#        a +=1

data_cleaned["City"] = data_cleaned["Location"].apply(lambda x: x.split(', ')[0])
data_cleaned["State"] = data_cleaned["Location"].apply(lambda x: x.split(',')[-1] if x.find(',') != 1 else "Remote") 
data_cleaned["State"] = data_cleaned["State"].apply(lambda x: x.strip() if x.strip().lower() != 'manhattan' else 'NY') # edge case, comment out if needed
data_cleaned["State"] = data_cleaned["State"].apply(lambda x: "PA" if x.strip().lower() == 'pennsylvania' else x)# edge case, comment out if needed
data_cleaned["State"] = data_cleaned["State"].apply(lambda x: "NY" if x.strip().lower() == 'new york state' else x)# edge case, comment out if needed
data_cleaned["State"] = data_cleaned["State"].apply(lambda x: "Remote" if x.strip().lower() == 'united states' else x)# edge case, comment out if needed
data_cleaned[['City','State']].value_counts().sort_values()


There are a couple of location inputs where the city is not given. I'll run through the dataframe and check for if the city == state OR if city != Remote and eliminate rows where this is true.

In [ ]:
data_cleaned = data_cleaned[(data_cleaned["City"]!=data_cleaned["State"])|(data_cleaned["City"]=="Remote")]
data_cleaned["State"].value_counts()

In [ ]:
data_cleaned.columns

In [105]:
# convert year founded to years in existance
currentyear = datetime.now().year
data_cleaned['Company Age (years)'] = data_cleaned["Founded"].astype(int).apply(lambda x:-1 if x==float(np.nan) else currentyear - x)
pd.pivot_table(data_cleaned,index=["State", "Job Title","Company Name"], values = "Salary Average",aggfunc='count').sort_values("State")

Salary Average
State  Job Title                                          Company Name                                           
AZ     Data Analyst                                       Solu Technology Partners                              5
       Data Analyst Specialist                            Infosys BPM Limited                                  19
       Investment Data Analyst                            Vanguard                                              4
CA     Senior Data Analyst (Local or Remote)              Point Digital Finance, Inc.                           1
       Research Data Analyst I - JC-310561                Department of Industrial Relations                    6
       Remote EDI Analyst Contractor                      CyberCoders                                           1
       Quality Data Analyst                               TikTok                                                9
       Junior Data Analyst/Data Steward                   MATRIX Resources                                     24
       Data Specialist                                    Kelly                                                 6
       Data Quality Analyst/Documentation Specialist      TargetCW                                             19
       Data Entry Analyst                                 Advantage Technical                                  36
       Data Analyst : 320637                              CorTech                                              19
       Research Data Analyst                              Stanford University                                   1
       Data Analyst                                       Premier Talent Partners                              21
                                                          Lakeshore Learning Materials, LLC                    24
                                                          Hulu                                                  3
                                                          Creative Circle                                      17
       Business Data Analyst                              Intuit - Data                                        11
       Business Analyst, Associate                        Blue Shield of California                             1
       Business Analyst Work                              BairesDev                                             2
       Business Analyst                                   Insight Global                                       11
       Assoc Data Analyst                                 Disney Streaming                                      7
       Data Analyst                                       Volt Workforce Solutions                             22
CO     Data Analyst                                       DISH - Corporate                                      1
                                                          Store Space Self Storage                              1
       Data Analyst I                                     Astreya                                               2
CT     Data Analyst                                       Insight Global                                       12
       Business Analyst                                   Vaco                                                 11
DE     Business Analyst                                   Kelly                                                 3
FL     Data Analyst                                       Spherion Gainesville, FL                              1
GA     Cybersecurity Analyst (Remote)                     Home Depot / THD                                      8
       Data Analyst                                       Seneca Resources                                      3
       Data and Reporting Analyst                         Insight Global                                        2
       Junior Business Analyst                            Genpact                                            

I see alot of duplicate entries for jobs under the same company/states. Since I don't want those duplicates to skew the results, I think it is best to run through the dataframe and remove them using pandas.**drop_duplicates**.

In [107]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 614 entries, 0 to 640
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Job Title            614 non-null    object 
 1   Salary Minimum       614 non-null    object 
 2   Salary Maximum       614 non-null    object 
 3   Salary Average       614 non-null    float64
 4   Rating               614 non-null    object 
 5   Company Name         614 non-null    object 
 6   Location             614 non-null    object 
 7   Size                 614 non-null    object 
 8   Founded              614 non-null    object 
 9   Type of ownership    614 non-null    object 
 10  Industry             614 non-null    object 
 11  Sector               614 non-null    object 
 12  Revenue              614 non-null    object 
 13  Average Hourly Rate  614 non-null    int64  
 14  City                 614 non-null    object 
 15  State                614 non-null    obj

In [108]:
data_drop_dups = data_cleaned.drop_duplicates()
data_drop_dups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 0 to 632
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Job Title            86 non-null     object 
 1   Salary Minimum       86 non-null     object 
 2   Salary Maximum       86 non-null     object 
 3   Salary Average       86 non-null     float64
 4   Rating               86 non-null     object 
 5   Company Name         86 non-null     object 
 6   Location             86 non-null     object 
 7   Size                 86 non-null     object 
 8   Founded              86 non-null     object 
 9   Type of ownership    86 non-null     object 
 10  Industry             86 non-null     object 
 11  Sector               86 non-null     object 
 12  Revenue              86 non-null     object 
 13  Average Hourly Rate  86 non-null     int64  
 14  City                 86 non-null     object 
 15  State                86 non-null     obje

In [110]:
pd.pivot_table(data_drop_dups,index=["State", "Job Title","Company Name"], values = "Salary Average",aggfunc='count').sort_values("State")

Salary Average
State  Job Title                                          Company Name                                           
AZ     Data Analyst                                       Solu Technology Partners                              1
       Data Analyst Specialist                            Infosys BPM Limited                                   1
       Investment Data Analyst                            Vanguard                                              1
CA     Senior Data Analyst (Local or Remote)              Point Digital Finance, Inc.                           1
       Research Data Analyst I - JC-310561                Department of Industrial Relations                    1
       Remote EDI Analyst Contractor                      CyberCoders                                           1
       Quality Data Analyst                               TikTok                                                1
       Junior Data Analyst/Data Steward                   MATRIX Resources                                      2
       Data Specialist                                    Kelly                                                 1
       Data Quality Analyst/Documentation Specialist      TargetCW                                              1
       Data Entry Analyst                                 Advantage Technical                                   1
       Data Analyst : 320637                              CorTech                                               1
       Research Data Analyst                              Stanford University                                   1
       Data Analyst                                       Premier Talent Partners                               1
                                                          Lakeshore Learning Materials, LLC                     1
                                                          Hulu                                                  1
                                                          Creative Circle                                       1
       Business Data Analyst                              Intuit - Data                                         1
       Business Analyst, Associate                        Blue Shield of California                             1
       Business Analyst Work                              BairesDev                                             1
       Business Analyst                                   Insight Global                                        1
       Assoc Data Analyst                                 Disney Streaming                                      1
       Data Analyst                                       Volt Workforce Solutions                              1
CO     Data Analyst                                       DISH - Corporate                                      1
                                                          Store Space Self Storage                              1
       Data Analyst I                                     Astreya                                               1
CT     Data Analyst                                       Insight Global                                        1
       Business Analyst                                   Vaco                                                  1
DE     Business Analyst                                   Kelly                                                 1
FL     Data Analyst                                       Spherion Gainesville, FL                              1
GA     Cybersecurity Analyst (Remote)                     Home Depot / THD                                      1
       Data Analyst                                       Seneca Resources                                      1
       Data and Reporting Analyst                         Insight Global                                        1
       Junior Business Analyst                            Genpact                                            

In [ ]:
# Group jobs under archetypes (Junior v senior, analyst v business analyst)
data_cleaned["Job Title"].value_counts() # count instances of job titles occuring

We can see from above that there are already some redundancies due to small changes in the titles (eg Sr. Data Analyst ). Let's group the jobs together with a function that searches the titles and combines everything under similar banners (manager, analyst, specialist, etc). Ken Jee created some nice functions that will serve us well, so if you wish to see more here is the link: https://youtu.be/QWgg4w1SpJ8.

In [114]:
#define functions to bin jobs into groups
def title_condencer(title):
    if 'scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'machine learning' in title.lower():
        return 'machine learning'
    elif 'data scientist' in title.lower():
        return 'data scientist'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'manager' in title.lower():
        return 'director'
    elif 'specialist' in title.lower():
        return 'specialist'
    elif 'business' in title.lower():
        return 'business-based'
    else:
        return 'Unbinned'

#identify if there is a seniority or level flag
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr.' in title.lower() or 'lead' in title.lower() or 'prinicpal' in title.lower() or 'iii' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'junior'
    elif 'business' in title.lower():
        return 'business'
    else:
        return 'na'

In [112]:
#Check binning
data_drop_dups['Job Title'] = data_drop_dups['Job Title'].values.astype(str)
data_drop_dups['Title Grouping'] = data_drop_dups['Job Title'].apply(title_condencer)
data_drop_dups['Title Grouping'].value_counts()

analyst       85
specialist     1
Name: Title Grouping, dtype: int64

In [115]:
#check seniority level
data_drop_dups['Seniority Level'] = data_drop_dups['Job Title'].apply(seniority)
data_drop_dups['Seniority Level'].value_counts()

na          67
business    10
junior       8
senior       1
Name: Seniority Level, dtype: int64

In [116]:
#input the filename you wish to save the information as
file = r"C:\Users\Tineash\Projects\Glassdoor_webscraper\Data\DA_data_cleaned_2.csv" #place the filepath between the quotes

data_drop_dups.to_csv(file, index = False)

In [ ]:
# check for string answers for Ownership column - to do

In [ ]:
# Replace string "unknown/Non-applicable" in revenue with NaN - to do

In [ ]:
# remove texts from revenue and convert revenue range to an average revenue as an int/float